<a href="https://colab.research.google.com/github/Ismail-therap/Apply-and-compare-ML-algorithm-on-Heart-Disease-Data-Set/blob/main/Predicting_Heart_disease_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

#######################################
### -------- Load libraries ------- #### Load Huggingface transformers

from transformers import TFBertModel,  BertConfig, BertTokenizerFast# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np

In [ ]:
import pandas as pd

url='https://drive.google.com/file/d/1t-t4fUA_tRl3nIS64zWpYQjqBB5QaiY8/view?usp=share_link'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(dwn_url)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,num,heart_patient
0,63,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,0,1
1,67,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,2,0
2,67,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,1,1
3,37,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,0,0
4,41,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,0,0


In [ ]:
df['text'] = df.iloc[:, :-1].astype(int).apply(lambda row: ''.join(map(str, row)), axis=1)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,num,heart_patient,text
0,63,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,0,1,63111452331215002300
1,67,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,2,0,67141602860210811232
2,67,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,1,1,67141202290212912221
3,37,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,0,0,37131302500018703300
4,41,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,0,0,41021302040217201100


In [ ]:
df_in_model = df[['text','heart_patient']]
df_in_model.head()

,text,heart_patient
0,63111452331215002300,1
1,67141602860210811232,0
2,67141202290212912221,1
3,37131302500018703300,0
4,41021302040217201100,0


In [ ]:
df_in_model, df_in_model_test = train_test_split(df_in_model, test_size = 0.2)

In [ ]:
#######################################
### --------- Setup BERT ---------- ###

# Name of the BERT model to use
model_name = 'bert-base-uncased'

# Max length of tokens
max_length = 14

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)

# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
#######################################
### ------- Build the model ------- ###

# TF Keras documentation: https://www.tensorflow.org/api_docs/python/tf/keras/Model

# Load the MainLayer
bert = transformer_model.layers[0]

# Build your model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
# attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32') 
# inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}
inputs = {'input_ids': input_ids}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)

# Then build your model output
heart_patient = Dense(units=len(df_in_model.heart_patient.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='heart_patient')(pooled_output)
#product = Dense(units=len(data.Product_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='product')(pooled_output)
outputs = {'heart_patient': heart_patient}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
model.summary()

Model: "BERT_MultiLabel_MultiClass"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 14)]              0         
                                                                 
 bert (TFBertMainLayer)      TFBaseModelOutputWithPoo  109482240 
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             14, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                        

In [ ]:
import tensorflow as tf
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.metrics import CategoricalAccuracy

# Set an optimizer
optimizer = tf.keras.optimizers.legacy.Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    clipnorm=1.0)

# Set loss and metrics
loss = {'heart_patient': BinaryCrossentropy(from_logits=True)}
metric = {'heart_patient': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metric)


In [ ]:
# Ready output data for the model
y_heart_patient = to_categorical(df_in_model['heart_patient'])

In [ ]:
# Tokenize the input (takes some time)
x = tokenizer(
    text=df_in_model['text'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
import time

start = time.time()

# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'heart_patient': y_heart_patient},
    validation_split=0.2,
    batch_size=100,
    epochs=10)

end = time.time()
execution_time = end - start

hours = int(execution_time // 3600)
minutes = int((execution_time % 3600) // 60)
seconds = execution_time % 60

print("Execution time: {:0>2}:{:0>2}:{:05.2f}".format(hours, minutes, seconds))



Epoch 1/10
2/2 [==============================] - 31s 12s/step - loss: 0.4387 - accuracy: 0.8355 - val_loss: 0.5275 - val_accuracy: 0.7436
Epoch 2/10
2/2 [==============================] - 27s 11s/step - loss: 0.4820 - accuracy: 0.8026 - val_loss: 0.5306 - val_accuracy: 0.7692
Epoch 3/10
2/2 [==============================] - 28s 12s/step - loss: 0.4173 - accuracy: 0.8355 - val_loss: 0.5685 - val_accuracy: 0.7179
Epoch 4/10
2/2 [==============================] - 27s 12s/step - loss: 0.3715 - accuracy: 0.8750 - val_loss: 0.5026 - val_accuracy: 0.7949
Epoch 5/10
2/2 [==============================] - 32s 16s/step - loss: 0.4065 - accuracy: 0.8618 - val_loss: 0.5411 - val_accuracy: 0.7692
Epoch 6/10
2/2 [==============================] - 30s 13s/step - loss: 0.3514 - accuracy: 0.8816 - val_loss: 0.5685 - val_accuracy: 0.7692
Epoch 7/10
2/2 [==============================] - 30s 13s/step - loss: 0.3000 - accuracy: 0.9145 - val_loss: 0.6366 - val_accuracy: 0.7179
Epoch 8/10
2/2 [===========